# Funciones de rendimiento espacial
_Autor:_    __Jesús Casado__ <br> _Revisión:_ __24/01/2020__

__Índice__ <br>
__[KGE](#KGE)__<br>
__[SPAEF](#SPAEF)__<br>
__[EOF](#EOF)__<br>

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
plt.style.use('dark_background')
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.gridspec import GridSpec

import seaborn as sns
sns.set()
custom_style = {'axes.facecolor': 'k',
                'axes.edgecolor': 'gray',
                'axes.labelcolor': 'white',
                'figure.facecolor': 'k',
                'grid.color': 'gray',
                'text.color': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white',
                "xtick.major.size": 0,
                "ytick.major.size": 0}
sns.set_style(style='darkgrid', rc=custom_style)

### KGE

In [3]:
def KGEsp(obs, sim, verbose=False, plot=False):
    """Calcula el coeficiente de eficiencia de Kling-Gupta entre dos mapas (observado y simulado).
    
            KGE = 1 - sqrt[(rho - 1)**2 + (alpha - 1)**2 + (beta - 1)**2]
            
    Donde 'rho' es el coeficiente de correlación de Pearson, 'alpha' es una medida de la variabilidad igual al cociente de las desviaciones típicas simulada y observada, y 'beta' es una medida del sesgo igual al cociente de las medias simulada y observada.
    
    Parámetros:
    -----------
    obs:       array (n,m). Mapa observado
    sim:       array (n,m). Mapa simulado
    verbose:   boolean. Si se quiere mostrar el proceso por pantalla
    plot:      boolean. Si se quieren mostrar gráficos
    
    Salida:
    -------
    kge:        float. Valor de la métrica"""
    
    # 'DataFrame' auxiliar (n,2); 'n' es el nº de celda con dato en ambos mapas
    aux = np.vstack((obs.flatten(), sim.flatten()))
    mask = np.any(np.isnan(aux), axis=0)
    df = pd.DataFrame(data=aux[:,~mask], index=['obs', 'sim']).T
    
    if plot == True:
        sns.jointplot(x='obs', y='sim', data=df, kind='reg')
    
    # coeficiente de correlación
    rho = df.corr().iloc[0,1]
    
    # cociente de las varianzas
    std = df.std()
    alpha = std.sim / std.obs
    # sesgo
    mean = df.mean()
    beta = mean.sim / mean.obs
    
    if verbose == True:
        print('rho = {0:.3f}\talpha = {1:.3f}\tbeta = {2:.3f}'.format(rho, alpha, beta))
        
    # coeficiente de eficiencia de Kling-Gupta
    kge = 1 - np.sqrt((rho - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
    if verbose == True:
        print('KGEsp = {0:.3f}'.format(kge))
    
    return kge

### SPAEF

In [1]:
def SPAEF(obs, sim, verbose=False, plot=False):
    """Calcula la métrica de rendimiento SPAEF (SPAtial EFicciency) entre dos mapas (observado y simulado).
    
            SPAEF = 1 - sqrt[(rho - 1)**2 + (gamma - 1)**2 + (delta - 1)**2]
            
    Dond 'rho' es el coeficiente de correlación de Pearson, 'gamma' es una medida relativa de la variabilidad igual al cociente entre los coeficientes de variación observados y simulados, y 'delta' es una medida de la variabilidad igual a la intersección entre los histogramas normalizados (N[0,1]) de los dos mapas.
    
    Parámetros:
    -----------
    obs:       array (n,m). Mapa observado
    sim:       array (n,m). Mapa simulado
    verbose:   boolean. Si se quiere mostrar el proceso por pantalla
    plot:      boolean. Si se quieren mostrar gráficos
    
    Salida:
    -------
    spaef:     float. Valor de la métrica"""
    
    # 'DataFrame' auxiliar (n,2); 'n' es el nº de celda con dato en ambos mapas
    aux = np.vstack((obs.flatten(), sim.flatten()))
    mask = np.any(np.isnan(aux), axis=0)
    df = pd.DataFrame(data=aux[:,~mask], index=['obs', 'sim']).T
    
    if plot == True:
        sns.jointplot(x='obs', y='sim', data=df, kind='reg')
    
    # COEFICIENTE DE CORRELACIÓN
    rho = df.corr().iloc[0,1]
    
    # COCIENTE DE LOS COEFICIENTES DE VARIACIÓN
    std = df.std()
    mean = df.mean()
    cv = std / mean
    gamma = cv.sim / cv.obs
    
    # INTERSECCIÓN DE LOS HISTOGRAMAS NORMALIZADOS
    # normalizar los datos
    Zscore = pd.DataFrame(index=df.index, columns=df.columns)
    for col in Zscore.columns:
        Zscore[col] = (df[col] - mean[col]) / std[col]
    # histogramas normalizados
    bins = np.arange(-5, 5.1, 0.25)
    KL = pd.DataFrame(index=bins[:-1], columns=Zscore.columns)
    for col in Zscore.columns:
        KL[col] = np.histogram(Zscore[col].values, bins=bins)[0]
    # calcular la intersección
    delta = KL.min(axis=1).sum() / KL.obs.sum()
    
    if verbose == True:
        print('rho = {0:.3f}\tgamma = {1:.3f}\tdelta = {2:.3f}'.format(rho, gamma, delta))
    
    if plot == True:
        plt.figure()
        sns.distplot(Zscore.obs, label='obs');
        sns.distplot(Zscore.sim, label='sim')
        plt.legend(fontsize=12);
    
    # CÁLCULO DE SPAEF
    spaef = 1 - np.sqrt((rho - 1)**2 + (gamma - 1)**2 + (delta - 1)**2)
    if verbose == True:
        print('SPAEF = {0:.3f}'.format(spaef))
    
    return spaef

### EOF